In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy

In [2]:
movies = pd.read_csv('movie_archive\movies_metadata.csv')
ratings_small = pd.read_csv('movie_archive\\ratings_small.csv')
links_small = pd.read_csv('movie_archive\links_small.csv')

C:\Users\Admin\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
word_bag = CountVectorizer(analyzer='word', ngram_range=(1, 3), stop_words='english')
bag_matrix = word_bag.fit_transform(movies['title'].apply(str) + " " + movies['overview'].apply(str))
bag_matrix

<45466x2407186 sparse matrix of type '<class 'numpy.int64'>'
	with 3943994 stored elements in Compressed Sparse Row format>

In [4]:
# not needed for creating user vectors

#cosine_similarity = cosine_similarity(bag_matrix, bag_matrix)

In [5]:
results = {}

#for idx, row in movies.iterrows():
    #print(idx)
    #similar_indices = cosine_similarity[idx].argsort()[:100:-1]
    #similar_items = [(cosine_similarity[idx][i], movies['id'][i]) for i in similar_indices]

    #results[row['id']] = similar_items[1:]

In [6]:
ratings_small.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [8]:
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [9]:
# changes type of all possible entrys in tmdbId to int, leaves rest in string
def list_str_to_int(s):     #überholt ----------
    if not s:
        return s
    try:
        i = int(s)
        return i
    except ValueError:
        return s

movieid_list = movies['id'].to_list()
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))
#movieid_list = list(map(list_str_to_int, movieid_list))


'1997-08-20' in movieid_list

False

In [10]:
userid_list = ratings_small['userId'].unique()
movieid_list = movies['id'].to_list()
# changes movieId type to int and deletes all false entrys
movieid_list = [x for x in movieid_list if x.isdigit()]
movieid_list = list(map(int, movieid_list))

def get_movie_vector(movieid):
    tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == movieid]
    try:
        index = movieid_list.index(int(tmdbId))
        movie_vector = bag_matrix[index]
        return movie_vector
    except ValueError:
        print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        return # <-------------------------------------????

# get all movie vector rated from user
def get_user_rated_movies(ids):
    #movie_vector_list = [get_movie_vector(i) for i in ids]
    
    movie_vector_list = []
    for i in ids:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == i]
        try:
            index = movieid_list.index(int(tmdbId))
            movie_vector = bag_matrix[index]
            movie_vector_list.append(movie_vector)
        except ValueError:
            print ("ValueError tmdbId: " + str(tmdbId))     # deleted/NaN movies
        
    movie_vectors = scipy.sparse.vstack(movie_vector_list)

    return movie_vectors

def create_user_vector(user_rated_movies_vector_list):
    user_vector = user_rated_movies_vector_list[0]
    for x in user_rated_movies_vector_list[1:]:
        user_vector = user_vector + x
    print("Time of Compute User Vector -> Normalize: ")
    %time user_norm = sklearn.preprocessing.normalize(user_vector)
    return sklearn.preprocessing.normalize(user_vector)

# compute all movie vector form specific user into one vector for each user
all_user_profiles = {}
for uid in userid_list:
    print("Time user_profile:")
    %time user_profile = ratings_small.loc[ratings_small['userId'] == uid]
    print("User-Profile Length: " + str(len(user_profile)))
    print("Time user_rated_movies_vector_list:")
    %time user_rated_movies_vector_list = get_user_rated_movies(user_profile['movieId'].to_list())
    #print(user_rated_movies_vector_list)
    #try:
        #user_vector = user_rated_movies_vector_list[0]
        #for x in user_rated_movies_vector_list[1:]:
        #    user_vector = user_vector + x

        #user_norm = sklearn.preprocessing.normalize(user_vector)
        #print(user_norm)

    print("Time Compute User Vector:")
    %time user_norm = create_user_vector(user_rated_movies_vector_list)

    all_user_profiles[uid] = user_norm
    print(str(uid) + " finished\n")
    #except TypeError:
    #    print(str(uid) + " didnt work, coo matrix parse error")

Time user_profile:
Wall time: 1.5 ms
User-Profile Length: 20
Time user_rated_movies_vector_list:
Wall time: 12.5 ms
Time Compute User Vector:
Time of Compute User Vector -> Normalize: 
Wall time: 499 µs
Wall time: 245 ms
1 finished

Time user_profile:
Wall time: 998 µs
User-Profile Length: 76
Time user_rated_movies_vector_list:
ValueError tmdbId: 607   NaN
Name: tmdbId, dtype: float64
Wall time: 31.5 ms
Time Compute User Vector:
Time of Compute User Vector -> Normalize: 
Wall time: 988 µs
Wall time: 1 s
2 finished

Time user_profile:
Wall time: 999 µs
User-Profile Length: 51
Time user_rated_movies_vector_list:
Wall time: 24.4 ms
Time Compute User Vector:
Time of Compute User Vector -> Normalize: 
Wall time: 810 µs
Wall time: 664 ms
3 finished

Time user_profile:
Wall time: 1.51 ms
User-Profile Length: 204
Time user_rated_movies_vector_list:
Wall time: 110 ms
Time Compute User Vector:
Time of Compute User Vector -> Normalize: 
Wall time: 1.5 ms
Wall time: 2.86 s
4 finished

Time user_pr

KeyboardInterrupt: 

In [15]:
# top recommendet movies for users
recommendation_users = {}

for uid in userid_list:
    user_vec = all_user_profiles[uid]
    cosine_similarity_user_movies = cosine_similarity(user_vec, bag_matrix)
    top_similar = cosine_similarity_user_movies[0].argsort()[:100:-1]
    recommendation_users[uid] = [(cosine_similarity_user_movies[0][i], movies['id'][i]) for i in top_similar]


In [16]:
len(recommendation_users)

671

In [34]:
#the first posts are always the already interacted ones ?!

percentage_already_seen = []
for uid in userid_list:
    user_profile = ratings_small.loc[ratings_small['userId'] == uid]
    user_recommendation = recommendation_users[uid]

    user_profile_mid_list = user_profile['movieId'].to_list()
    user_tmdb_list = []
    for m in user_profile_mid_list:
        tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
        try:
            user_tmdb_list.append(int(tmdbId))
        except ValueError:
            print("ValueError: " + str(tmdbId))
    
    m_count = 0
    for movie_cosine, mid in user_recommendation[:len(user_profile)]:
        if int(mid) in user_tmdb_list:
            m_count += 1
            

    if m_count == len(user_profile):
        print(str(uid) + " : the first movies are already viewed")
    else:
        print(str(m_count) + " of " + str(len(user_profile)) + " in first recommended already seen")

    p = m_count/len(user_profile)
    percentage_already_seen.append(p)

average_percentage = sum(percentage_already_seen) / len(percentage_already_seen)
print("Average Percentage of already seen movies: " + str(average_percentage))

    


ror: 8045   NaN
Name: tmdbId, dtype: float64
ValueError: 8507   NaN
Name: tmdbId, dtype: float64
24 of 676 in first recommended already seen
44 of 114 in first recommended already seen
31 of 33 in first recommended already seen
26 of 183 in first recommended already seen
37 of 39 in first recommended already seen
267 : the first movies are already viewed
29 of 400 in first recommended already seen
24 of 25 in first recommended already seen
ValueError: 8045   NaN
Name: tmdbId, dtype: float64
ValueError: 8505   NaN
Name: tmdbId, dtype: float64
30 of 229 in first recommended already seen
65 of 72 in first recommended already seen
28 of 39 in first recommended already seen
51 of 92 in first recommended already seen
37 of 39 in first recommended already seen
31 of 202 in first recommended already seen
16 of 21 in first recommended already seen
27 of 58 in first recommended already seen
35 of 53 in first recommended already seen
50 of 57 in first recommended already seen
280 : the first movi

In [37]:
average_percentage

0.6111864613910986

In [16]:
user_profile = ratings_small.loc[ratings_small['userId'] == 1]

In [38]:
user_profile
test = recommendation_users[1]
movie_cosine, mid = test[0]
user_profile_mid_list = user_profile['movieId'].to_list()
user_tmdb_list = []
for m in user_profile_mid_list:
    tmdbId = links_small['tmdbId'].loc[links_small['movieId'] == m]
    try:
        user_tmdb_list.append(int(tmdbId))
    except ValueError:
        print("ValueError: " + str(tmdbId))
print(mid)
movie_title = movies['title'].loc[movies['id'] == mid]
print(movie_title)
user_tmdb_list



665
1242    Ben-Hur
Name: title, dtype: object


[862,
 687,
 629,
 11929,
 11,
 680,
 278,
 13,
 712,
 11310,
 5503,
 14291,
 9479,
 812,
 280,
 581,
 275,
 532,
 630,
 15121,
 562,
 583,
 792,
 762,
 531,
 1891,
 2493,
 85,
 185,
 525,
 530,
 279,
 218,
 37247,
 235,
 137,
 33,
 89,
 578,
 1669,
 9427,
 4995,
 563,
 489,
 544,
 165,
 857,
 1779,
 117,
 162,
 637,
 9487,
 10162,
 1934,
 8879,
 1542,
 603,
 817,
 745,
 11153,
 2280,
 850,
 14,
 9377,
 550,
 492,
 1832,
 11663,
 863,
 497,
 334,
 8872,
 9609,
 820,
 8469,
 243,
 11072,
 7443,
 786,
 146,
 107,
 711,
 8358,
 134,
 11973,
 1900,
 808,
 824,
 11457,
 585,
 671,
 161,
 194,
 120,
 453,
 855,
 425,
 8346,
 557,
 245,
 180,
 4147,
 1430,
 672,
 2757,
 121,
 640,
 1574,
 423,
 10678,
 25461,
 51927,
 604,
 1088,
 4464]

In [27]:
 import pickle

In [28]:
ur_file = open('small_user_recommednations.dat', 'wb')

In [29]:
pickle.dump(recommendation_users, ur_file)
ur_file.close()

In [26]:
len(recommendation_users)

671

In [22]:
type(recommendation_users)

dict